In [122]:
import pandas as pd
import sys
sys.path.append('/ref/analysis/pipelines/')
import kang
import re
from tqdm import tqdm
import random

In [2]:
file_ath_pep = '/ref/analysis/References/Athaliana/annotation/Athaliana_167_TAIR10.protein_primaryTranscriptOnly.fa'

In [3]:
dic_ath_pep = kang.Fasta2dic(file_ath_pep)

In [4]:
# number of protein in total
len(dic_ath_pep.keys())

27416

In [5]:
aminoacids_pre = set(kang.gencode.values())

In [6]:
aminoacids = list(aminoacids_pre-{'*'})

In [149]:
comp = []
for aa in aminoacids:
    for aa1 in aminoacids:
        comp.append(aa+aa1)
        #comp.append(aa+'.'+aa1)

In [150]:
test = dic_ath_pep.values()[3]
test2 = dic_ath_pep.values()[4]

In [151]:
key_list = []
for c1 in tqdm(comp):
    for c2 in comp: 
        key = [c1,c2]
        key.sort()
        key = tuple(key)
        key_list.append(key)   

100%|██████████| 400/400 [00:00<00:00, 2485.98it/s]


In [152]:
key_list_rd = list(set(key_list))

In [139]:
def get_score_array(seq1,seq2):
    score =[]
    for a,b in key_list_rd:
        s_c1 = bool(re.search(a,seq1))
        s_c2 = bool(re.search(b,seq2))
        if s_c1 + s_c2 == 2:
            score.append(1)
        elif s_c1 + s_c2 < 2:
            score.append(0)
        else:
            print s_c1,s_c2
    return(score)

    

----

In [107]:
# Database intact parsing
file_intact = './intact.ath.txt' 
df_intact_pre = pd.read_csv(file_intact,sep='\t')

mask = df_intact_pre['Host organism(s)'].apply(lambda x : 'taxid:3702' in x) # 순수 아라비돕시스 결과만 가져온다. 
df_intact = df_intact_pre[mask]

def get_genename(x):
    value = [a.split('(')[0] for a in x.split('|')]
    key   = [a.split('(')[1].replace(')','') for a in x.split('|')]
    dic = dict(zip(key,value))
    try:
        #print dic['locus name']
        result = re.search('AT.G[0-9]+',dic['locus name'].upper()).group(0)
        return result
    except:
        return None
              
df_intact['genename_A']=df_intact['Alias(es) interactor A'].apply(get_genename)
df_intact['genename_B']=df_intact['Alias(es) interactor B'].apply(get_genename)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [108]:
intact_positives = []
for n,genenameA in enumerate(df_intact['genename_A']):
    genenameB = list(df_intact['genename_B'])[n]
    key = [genenameA,genenameB]
    key.sort()
    key = tuple(key)
    intact_positives.append(key)
intact_positives = set(intact_positives)

In [109]:
# number of edges in intact
len(intact_positives)

2858

In [110]:
len(df_intact_pre) # 요거 고쳐라. 

19028

----

In [103]:
# AI1 parsing
file_ai1 = './ai1.tableS4.txt'
df_ai1   = pd.read_csv(file_ai1,sep='\t')

In [111]:
mask = df_ai1['AI-1MAIN'] == 1
df_ai1_pos = df_ai1[mask]

In [116]:
ai1_positives = []
for n,genenameA in enumerate(df_ai1_pos['TAIR_LOCUS_IDA']):
    genenameB = list(df_ai1_pos['TAIR_LOCUS_IDB'])[n]
    key = [genenameA,genenameB]
    key.sort()
    key = tuple(key)
    ai1_positives.append(key)

In [131]:
negative_references = []
while len(negative_references) != len(ai1_positives):
    genenameA = random.choice(dic_ath_pep.keys()).split('.')[0]
    genenameB = random.choice(dic_ath_pep.keys()).split('.')[0]
    key = [genenameA,genenameB]
    key.sort()
    key = tuple(key)
    if key in set(ai1_positives) or key in set(intact_positives):
        pass
    else:
        negative_references.append(key)
    

In [135]:
set(negative_references) & set(intact_positives)

set()

In [147]:
key   = [x.split('.')[0] for x in dic_ath_pep.keys()]
value = dic_ath_pep.values()
dic_ath_pep_genename = dict(zip(key,value))

In [ ]:
dicGP2Score = {}
for a,b in tqdm(ai1_positives):
    try:
        seq1 = dic_ath_pep_genename[a]
        seq2 = dic_ath_pep_genename[b]
        dicGP2Score[(a,b)] = get_score_array(seq1,seq2)
        dicGP2Score[(a,b)].append(1)
    except KeyError:
        continue

 65%|██████▍   | 3677/5664 [3:59:27<2:10:38,  3.94s/it]

In [163]:
for a,b in tqdm(negative_references[0:100]):
    try:
        seq1 = dic_ath_pep_genename[a]
        seq2 = dic_ath_pep_genename[b]
        dicGP2Score[(a,b)] = get_score_array(seq1,seq2)
        dicGP2Score[(a,b)].append(0)
    except KeyError:
        continue

100%|██████████| 100/100 [06:35<00:00,  4.03s/it]


In [161]:
len(dicGP2Score.keys())

5632

In [162]:
len(ai1_positives)

5664

In [144]:
len(key_list_rd)

320400